In [ ]:
"""
Pipeline prático (Etapa 1): comparar timbre entre 5 instrumentos


Objetivo: validar na prática a ideia de que timbre diferencia sons com mesma nota, duração e intensidade.


Como usar:
1) Preencha os caminhos dos 4 áudios (≈5s cada) nas variáveis abaixo (guitarra, piano, flauta, bateria).
- Idealmente: faixas monofônicas com a mesma nota (ex.: A4 = 440 Hz), volume semelhante e sem efeitos.
- Caso não sejam iguais, o script normaliza o nível (RMS matching) e recorta/ajusta duração.
2) Execute o script inteiro. Ele irá:
- Carregar e padronizar os áudios (mono, 22050 Hz, duração comum).
- Visualizar waveform, envelope e espectrograma de cada instrumento.
- Extrair um conjunto de features (tempo, espectrais, perceptuais) e montar um DataFrame
com médias/descritores por arquivo.
- Plotar comparações chave (ex.: centroid, bandwidth, flux, flatness, ZCR, HNR aproximado) e
salvar um CSV com as métricas agregadas.


Dependências:
- librosa, numpy, pandas, matplotlib, scipy


Observação:
- Métricas como "ataque" e "HNR" aqui são aproximações simples para a etapa 1.
Podemos sofisticar depois (ex.: detecção de onsets/ADSR, HNR via cepstrum, etc.).
"""

'\nPipeline prático (Etapa 1): comparar timbre entre 5 instrumentos\n\n\nObjetivo: validar na prática a ideia de que timbre diferencia sons com mesma nota, duração e intensidade.\n\n\nComo usar:\n1) Preencha os caminhos dos 4 áudios (≈5s cada) nas variáveis abaixo (guitarra, piano, flauta, bateria).\n- Idealmente: faixas monofônicas com a mesma nota (ex.: A4 = 440 Hz), volume semelhante e sem efeitos.\n- Caso não sejam iguais, o script normaliza o nível (RMS matching) e recorta/ajusta duração.\n2) Execute o script inteiro. Ele irá:\n- Carregar e padronizar os áudios (mono, 22050 Hz, duração comum).\n- Visualizar waveform, envelope e espectrograma de cada instrumento.\n- Extrair um conjunto de features (tempo, espectrais, perceptuais) e montar um DataFrame\ncom médias/descritores por arquivo.\n- Plotar comparações chave (ex.: centroid, bandwidth, flux, flatness, ZCR, HNR aproximado) e\nsalvar um CSV com as métricas agregadas.\n\n\nDependências:\n- librosa, numpy, pandas, matplotlib, sci

In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
import librosa
import librosa.display
import matplotlib.pyplot as plt
from scipy.signal import hilbert

In [ ]:
SR = 22050 # target sample rate
TARGET_DUR = 5.0 # target duration in seconds
ROLL_PERCENT = 0.85 # roll-off
N_MFCC = 13


# >>>> Substitua pelos caminhos dos seus áudios <<<<
AUDIO_PATHS = {
"piano": "../data/piano.wav",
"flute": "../data/flute.wav",
"trumpet": "../data/trumpet.wav",
"drums": "../data/drums.wav",
"bass": "../data/bass.wav",
}


OUT_DIR = Path("../output")
OUT_DIR.mkdir(parents=True, exist_ok=True)

In [3]:
def load_pad_mono(path: str, sr: int = SR, target_dur: float = TARGET_DUR):
    y, _sr = librosa.load(path, sr=sr, mono=True)
    # pad/truncate to target_dur
    target_n = int(target_dur * sr)
    if len(y) < target_n:
        y = np.pad(y, (0, target_n - len(y)))
    else:
        y = y[:target_n]
    return y




def rms(x):
    return np.sqrt(np.mean(x**2) + 1e-12)




def match_rms(y, ref_level: float = None):
    """Normaliza o nível RMS. Se ref_level None, usa -20 dBFS como alvo aproximado."""
    if ref_level is None:
        ref_level = 10 ** (-20/20) # -20 dBFS
    y_rms = rms(y)
    if y_rms < 1e-9:
        return y
    gain = ref_level / y_rms
    return np.clip(y * gain, -1.0, 1.0)




def temporal_envelope(y):
# Envelope pelo sinal analítico (Hilbert)
    analytic = hilbert(y)
    env = np.abs(analytic)
    return librosa.util.normalize(env)




def estimate_attack_time(env, sr=SR, floor_db=-60, thresh_db=-3):
    """Aproximação: tempo até o envelope atingir (max + thresh_db).
    floor_db evita valores muito baixos; resultado em segundos."""
    env_db = librosa.amplitude_to_db(env + 1e-9, ref=np.max)
    # ponto onde cruza -3 dB do pico
    idx = np.argmax(env_db >= thresh_db)
    return idx / sr




def spectral_features(y, sr=SR):
    S = np.abs(librosa.stft(y, n_fft=2048, hop_length=512))
    S_power = S**2
    spec_centroid = librosa.feature.spectral_centroid(S=S, sr=sr)
    spec_bw = librosa.feature.spectral_bandwidth(S=S, sr=sr)
    spec_roll = librosa.feature.spectral_rolloff(S=S, sr=sr, roll_percent=ROLL_PERCENT)
    spec_flat = librosa.feature.spectral_flatness(S=S)
    spec_contrast = librosa.feature.spectral_contrast(S=S, sr=sr)
    spec_flux = librosa.onset.onset_strength(S=librosa.power_to_db(S_power, ref=np.max))
    zcr = librosa.feature.zero_crossing_rate(y)
    return {
    "centroid": spec_centroid,
    "bandwidth": spec_bw,
    "rolloff": spec_roll,
    "flatness": spec_flat,
    "contrast": spec_contrast,
    "flux": spec_flux[None, :], # forma (1, T) para uniformizar
    "zcr": zcr,
    }




def perceptual_features(y, sr=SR, n_mfcc=N_MFCC):
    S = np.abs(librosa.stft(y, n_fft=2048, hop_length=512))
    mel = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=64)
    mfcc = librosa.feature.mfcc(S=librosa.power_to_db(mel, ref=np.max), sr=sr, n_mfcc=n_mfcc)
    chroma = librosa.feature.chroma_stft(S=S, sr=sr)
    # tempo (aprox) e taxa de onsets
    tempo, beats = librosa.beat.beat_track(y=y, sr=sr)
    onset_env = librosa.onset.onset_strength(y=y, sr=sr)
    onset_rate = librosa.onset.onset_detect(onset_envelope=onset_env, sr=sr)
    return {
    "mfcc": mfcc,
    "chroma": chroma,
    "tempo": np.array([[tempo]]),
    "onset_rate": np.array([[len(onset_rate) / (len(y)/sr + 1e-9)]]),
    }




def summarize_feature(name, mat):
    """Gera estatísticas simples por feature (média, desvio, mediana).
    mat esperado shape (F, T) ou (1, T)."""
    mat = np.atleast_2d(mat)
    feats = {}
    feats[f"{name}_mean"] = float(np.mean(mat))
    feats[f"{name}_std"] = float(np.std(mat))
    feats[f"{name}_median"] = float(np.median(mat))
    return feats




def summarize_vector(name, vec):
    vec = np.asarray(vec).ravel()
    return {
        f"{name}_mean": float(np.mean(vec)),
        f"{name}_std": float(np.std(vec)),
        f"{name}_median": float(np.median(vec)),
    }




def hnr_proxy(y, sr=SR):
    """Proxy grosseiro para Harmonic-to-Noise Ratio usando autocorrelação.
    Retorna valor em dB (quanto maior, mais harmônico)."""
    y = y - np.mean(y)
    if np.max(np.abs(y)) > 0:
        y = y / np.max(np.abs(y))
    ac = librosa.autocorrelate(y)
    ac = ac / (np.max(ac) + 1e-9)
    # pular lag 0; pegar pico maior em [2ms, 20ms] ~ [1100Hz, 50Hz]
    minlag = int(sr/500) # 2 ms
    maxlag = int(sr/50) # 20 ms
    peak = np.max(ac[minlag:maxlag]) if maxlag > minlag else 0
    noise = np.mean(ac[maxlag:]) if len(ac) > maxlag else 1e-9
    hnr = 10*np.log10((peak+1e-9)/(noise+1e-9))
    return hnr

In [4]:
waves = {}
for inst, path in AUDIO_PATHS.items():
    p = Path(path)
    if not p.exists():
        print(f"[AVISO] Arquivo não encontrado: {p}. Pule ou ajuste o caminho.")
        continue
    y = load_pad_mono(str(p))
    y = match_rms(y)
    waves[inst] = y


# Visualizações por instrumento
for inst, y in waves.items():
    t = np.arange(len(y))/SR
    env = temporal_envelope(y)


    # Waveform + envelope
    plt.figure(figsize=(10, 3))
    plt.title(f"Waveform + Envelope — {inst}")
    plt.plot(t, y, linewidth=0.8)
    plt.plot(t, env * np.max(np.abs(y)), linewidth=1.0)
    plt.xlabel("Tempo (s)")
    plt.ylabel("Amplitude")
    plt.tight_layout()
    plt.savefig(OUT_DIR / f"{inst}_wave_env.png", dpi=150)
    plt.close()


    # Espectrograma (mel)
    mel = librosa.feature.melspectrogram(y=y, sr=SR, n_mels=96)
    mel_db = librosa.power_to_db(mel, ref=np.max)
    plt.figure(figsize=(10, 3))
    plt.title(f"Mel-Spectrogram — {inst}")
    librosa.display.specshow(mel_db, sr=SR, x_axis='time', y_axis='mel')
    plt.colorbar(format='%+2.0f dB')
    plt.tight_layout()
    plt.savefig(OUT_DIR / f"{inst}_melspec.png", dpi=150)
    plt.close()

In [5]:
rows = []
for inst, y in waves.items():
    env = temporal_envelope(y)
    attack_s = estimate_attack_time(env, sr=SR)


    sf = spectral_features(y)
    pf = perceptual_features(y)
    proxy_hnr = hnr_proxy(y)


    feats = {"instrumento": inst,
        "rms": rms(y),
        "attack_time_s": attack_s,
        "hnr_proxy_db": proxy_hnr}


    # Resumos simples
    for k in ["centroid", "bandwidth", "rolloff", "flatness", "flux", "zcr"]:
        feats.update(summarize_feature(k, sf[k]))
    feats.update(summarize_feature("contrast", sf["contrast"]))


    # MFCC: média/variância por coeficiente + médias globais
    mfcc = pf["mfcc"] # (n_mfcc, T)
    for i in range(mfcc.shape[0]):
        feats[f"mfcc{i+1}_mean"] = float(np.mean(mfcc[i]))
        feats[f"mfcc{i+1}_std"] = float(np.std(mfcc[i]))
    feats["mfcc_global_mean"] = float(np.mean(mfcc))
    feats["mfcc_global_std"] = float(np.std(mfcc))


    # Chroma: média por classe de pitch
    chroma = pf["chroma"] # (12, T)
    for i in range(12):
        feats[f"chroma{i}_mean"] = float(np.mean(chroma[i]))


    # Ritmo
    feats["tempo_est_bpm"] = float(pf["tempo"][0,0])
    feats["onset_rate_s"] = float(pf["onset_rate"][0,0])


    rows.append(feats)


    feat_df = pd.DataFrame(rows).set_index("instrumento")
    feat_df.to_csv(OUT_DIR / "features_resumo.csv", index=True)
    print(f"CSV salvo em: {OUT_DIR / 'features_resumo.csv'}")

/tmp/ipykernel_75408/3239046703.py:132: RuntimeWarning: invalid value encountered in log10
  hnr = 10*np.log10((peak+1e-9)/(noise+1e-9))
/tmp/ipykernel_75408/1761943204.py:40: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  feats["tempo_est_bpm"] = float(pf["tempo"][0,0])


CSV salvo em: outputs_features_etapa1/features_resumo.csv
CSV salvo em: outputs_features_etapa1/features_resumo.csv
CSV salvo em: outputs_features_etapa1/features_resumo.csv
CSV salvo em: outputs_features_etapa1/features_resumo.csv
CSV salvo em: outputs_features_etapa1/features_resumo.csv


In [6]:
metrics_to_plot = [
    ("centroid_mean", "Centroid (média)"),
    ("bandwidth_mean", "Bandwidth (média)"),
    ("rolloff_mean", f"Roll-off {int(ROLL_PERCENT*100)}% (média)"),
    ("flatness_mean", "Flatness (média)"),
    ("flux_mean", "Spectral Flux (média)"),
    ("zcr_mean", "Zero-Crossing Rate (média)"),
    ("hnr_proxy_db", "HNR Proxy (dB)"),
    ("attack_time_s", "Tempo de ataque (s)"),
]


for key, label in metrics_to_plot:
    if key not in feat_df.columns:
        continue
    plt.figure(figsize=(7, 4))
    feat_df[key].plot(kind="bar")
    plt.title(label)
    plt.ylabel(label)
    plt.tight_layout()
    plt.savefig(OUT_DIR / f"compare_{key}.png", dpi=150)
    plt.close()


print("Concluído. Verifique a pasta outputs_features_etapa1 com imagens e CSV.")

Concluído. Verifique a pasta outputs_features_etapa1 com imagens e CSV.
